In [0]:
%pip install mlflow==2.20.2 databricks-sdk==0.40.0

import os
import mlflow
from mlflow.store.artifact.models_artifact_repo import ModelsArtifactRepository
mlflow.set_registry_uri('databricks-uc')

local_path = ModelsArtifactRepository(f"models:/iot_turbine.dev.turbine_maintenance@prod").download_artifacts("")
requirements_path = os.path.join(local_path, "requirements.txt")

%pip install -r $requirements_path
dbutils.library.restartPython()

#Registering python UDF to a SQL function
This is a companion notebook to load the wind turbine prediction model as a spark udf and save it as a SQL function
 
Make sure you add this notebook in your DLT job to have access to the `get_turbine_status` function. (Currently mixing python in a SQL DLT notebook won't run the python)

In [0]:
import mlflow
mlflow.set_registry_uri('databricks-uc')

predict_maintenance_udf = mlflow.pyfunc.spark_udf(spark,
                                                  "models:/iot_turbine.dev.turbine_maintenance@prod",
                                                  result_type="string")

In [0]:
spark.udf.register("predict_maintenance", predict_maintenance_udf)